Think Stats book - Chapter 6: Operations on Distributions

In [138]:
import math
from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from scipy.stats import norm, binom, skew

Skewness is a statistic that measures the asymmetry of a distribution. Negative skewness indicates that a distribution extends farther to the left than the right. Positive skewness indicates the opposite. Since it's sensitive to outliers, it's usually not a good idea to compute skewness. Besides, there are another ways to evaluate the asymmetry of a distribution such as by looking at the relationship between the mean and the median. Pearson's median skewness coefficient is an alternative way to compute the skewness.

In [102]:
#Compute the skewness for the distributions of pregnancy length and birth weight.
def skewness(x):
    x = np.array(x)
    mean = np.mean(x)
    variance = np.mean(np.power((x - mean), 2)) #also known as mean squared deviation; np.var(x) 
    mean_cubed_deviation = np.mean(np.power((x - mean), 3))
    skewness = mean_cubed_deviation / math.pow(variance, 1.5)
    
    return skewness

def pearson_skewness(x):
    x = np.array(x)
    mean = np.mean(x)
    median = np.median(x)
    std = np.std(x)
    pearson_skewness = 3 * (mean - median) / std
    
    return pearson_skewness

    
data = pd.read_fwf("2002FemPreg.dat", names=["caseid", "nbrnaliv", "babysex", "birthwgt_lb","birthwgt_oz", 
                                             "prglength", "outcome", "birthord", "agepreg", "finalwgt"],
                                      colspecs=[(0, 12), (21, 22), (55, 56), (57, 58), (58, 60),
                                                (274, 276), (276, 277), (278, 279), (283, 285), (422, 439)])
data.head()

,caseid,nbrnaliv,babysex,birthwgt_lb,birthwgt_oz,prglength,outcome,birthord,agepreg,finalwgt
0,1,1.0,1.0,8.0,13.0,39,1,1.0,33.0,6448.271112
1,1,1.0,2.0,7.0,14.0,39,1,2.0,39.0,6448.271112
2,2,3.0,1.0,9.0,2.0,39,1,1.0,14.0,12999.542264
3,2,1.0,2.0,7.0,0.0,39,1,2.0,17.0,12999.542264
4,2,1.0,2.0,6.0,3.0,39,1,3.0,18.0,12999.542264


In [103]:
live_births = data[data["outcome"] == 1]
birth_weight = (live_births['birthwgt_lb']*16 + live_births['birthwgt_oz']).dropna()

print(skew(birth_weight))
print(skewness(birth_weight))
print(pearson_skewness(birth_weight))

-1.3179155263393318
-1.3179155263393318
-0.3824623067640262


In [104]:
pregnancy_length = live_births["prglength"].dropna()

print(skew(pregnancy_length))
print(skewness(pregnancy_length))
print(pearson_skewness(pregnancy_length))

-2.855300690938406
-2.8553006909384067
-0.48787019646027696


Random Variables: A random variable represents a process that generates a random number. When you see a random variable, you should think "a value selected from a distribution". You may think of a random variable as an object that provides a method(let's call it "generate") that uses a random process to generate values.

CDF_X(x) = p(X <= x), the CDF of the random variable X, evaluated for a particular value x, is defined as the probability that a value generated by the random process X is less than or equal to x.

Example 6-4. Write a definition for a class that represents a random variable with a Gumbel distribution. The Gumbel distribution is used to model the distribution of the maximum (or the minimum) of a number of samples of various distributions. The potential applicability of the Gumbel distribution to represent the distribution of maxima relates to extreme value theory, which indicates that it is likely to be useful if the distribution of the underlying sample data is of the normal or exponential type. 

In [120]:
#Q(p) = mu - beta*ln(-ln(p)), where p is drawn from the uniform(family of symmetric probability distributions) 
#distribution on the inverval (0,1)

class RandomVariable(object):
    """ Parent class of all random variables """
    
class Exponential(RandomVariable):
    def __init__(self, lam):
        self.lam = lam
    def generate(self):
        return np.random.exponential(self.lam)

class Gumbel(RandomVariable):
    def __init__(self, mu, beta):
        self.mu = mu
        self.beta = beta
    def generate(self):
        return self.mu - (self.beta * math.log(-math.log(np.random.random(), math.e), math.e))

    
gumbel = Gumbel(0.01, 1)
print(gumbel.generate())
exponential = Exponential(0.7)
print(exponential.generate())

0.7720881098219216
0.48377712963662145


PDFs: The derivative of a CDF is called a Probability Density Function(PDF). Remember that PDF doesn't represent probability, it represents probability density(or probability per unit of x). So, in order to get a probability mass you have to integrate over x.    

Example 6-6. In the BRFSS, the distribution of heights is roughly normal with parameters mean = 178cm and variance = 59.4cm for men, and mean = 163cm and variance = 52.8cm for women. In order to join Blue Man Group, you have to be male between 155.4cm and 189.0cm. What percentage of the U.S male population is in this range?

In [143]:
men_mean = 178.0
men_variance = 59.4
max_from_range = 189.0
min_from_range = 155.4

norm = stats.norm(loc=men_mean, scale=math.sqrt(men_variance))
probability = norm.cdf(max_from_range) - norm.cdf(min_from_range)
print(probability)

0.9215637391234368


Suppose we have two random variables, X and Y, with distributions CDF_X and CDF_Y. A random variable Z is defined as Z = X+Y. The convolution of probability distributions arises in probability theory and statistics as the operation in terms of probability distributions that corresponds to the addition of independent random variables and, by extension, to forming linear combinations of random variables. The operation here is a special case of convolution in the context of probability distributions.

Normal Distributions are amenable to analysis, specially because they are closed under linear transformation and convolution. 

If we add values drawn from normal distribututions, the distribution of the sum is normal. If we add values drawn from other distributions, the sum doesn't generally have one of the continuous distributions we have seen.

Central Limit Theorem: If we add up a large number of values from almost any distribution, the distribution of the sum converges to normal. Caveats: the values have to be drawn independently, the values have to come from the same distribution(although this requirement can be relaxed), the values have to be drawn from a distribution with finite mean and variance(so most Pareto distributions are out), the number of values you need before you see convergence depends on the skewness of the distribution. The Central Limit Theorem explains the prevalence of normal distributions in the real world.

The Distribution Framework: How does CDF, PMF and PDF relate to each other?

CDF -> differentiate -> PDF (both are continuous; pdf is non-cumulative and cdf is cumulative)

PDF -> integrate -> CDF

PDF -> bin -> PMF (both are non-cumulative; pdf is continuous and pmf is discrete)

PMF -> sum -> CDF (both are discrete - remember CDF might be either discrete or continuous; cdf is cumulative and pmf is non-cumulative)

CDF(discrete) -> diff -> PMF

CDF(discrete) -> smooth -> CDF(continuous) 